In [1]:
pip install pillow torch diffusers google-cloud-storage langchain pinecone-client google-generativeai serpapi google-colab > /dev/null 2>&1


In [5]:
!pip install pinecone-client > /dev/null 2>&1
!pip install tiktoken > /dev/null 2>&1
!pip install google-search-results > /dev/null 2>&1
!pip install -U langchain langchain-community > /dev/null 2>&1



In [18]:
# Install necessary libraries

# Import necessary modules
import uuid
import os
import json
from PIL import Image, ImageDraw, ImageFont
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from google.cloud import storage
from langchain.embeddings.openai import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
import google.generativeai as genai
from google.colab import userdata
from serpapi.google_search import GoogleSearch
import logging


# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Set up secrets
MONGO_URI = userdata.get("MONGO_URI")
PINE_CONE_API = userdata.get("PINE_CONE_API")
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
OPEN_AI_APIKEY = userdata.get("OPEN_AI_APIKEY")
SERPER_API_KEY = userdata.get("SERPER_API_KEY")

# Set the Google Application Credentials environment variable
GCP_CREDENTIALS_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS", "/content/drive/MyDrive/service_account_gcp/encoded-blend-439905-g7-3deac22ca1bf.json")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GCP_CREDENTIALS_PATH

# Initialize Generative AI API
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize Google Cloud Storage
GCS_BUCKET_NAME = "fashion_designer"
storage_client = storage.Client()
bucket = storage_client.bucket(GCS_BUCKET_NAME)

def upload_to_gcs(file_path, blob_name):
    """Uploads a file to GCS and returns the public URL."""
    try:
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(file_path)
        return blob.public_url
    except Exception as e:
        logging.error(f"Error uploading to GCS: {e}")
        raise

# Directories for saving images
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load Stable Diffusion model
def load_model():
    try:
        logging.info("Loading Stable Diffusion model...")
        model_path = "CompVis/stable-diffusion-v1-4"
        device = "cuda" if torch.cuda.is_available() else "cpu"
        precision = torch.float16 if device == "cuda" else torch.float32
        pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=precision).to(device)
        pipe.enable_attention_slicing()
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
        logging.info(f"Stable Diffusion loaded on {device}.")
        return pipe
    except Exception as e:
        logging.error(f"Failed to load Stable Diffusion model: {e}")
        raise

pipe = load_model()

# Initialize Pinecone
def init_pinecone():
    try:
        logging.info("Initializing Pinecone...")
        pinecone_client = Pinecone(api_key=PINE_CONE_API)
        return pinecone_client
    except Exception as e:
        logging.error(f"Failed to initialize Pinecone: {e}")
        raise

pinecone_client = init_pinecone()
male_index_name = "fashion"
female_index_name = "fashionfemale"

# Create indexes if they don't exist
for index_name in [male_index_name, female_index_name]:
    if index_name not in [idx.name for idx in pinecone_client.list_indexes()]:
        pinecone_client.create_index(
            name=index_name,
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )

# Load indexes
male_index = pinecone_client.Index(male_index_name)
female_index = pinecone_client.Index(female_index_name)

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(api_key=OPEN_AI_APIKEY)

def refine_prompt_with_gemini(user_prompt, gender, age):
    """Refines the user prompt using Gemini."""
    prompt = f"""
    Refine the user input to describe a unique, stylish dress with specific details for Stable Diffusion model; keep it under 20 words:
    User input: "{user_prompt}" for the gender : {gender} and this {age}.
    """
    try:
        model = genai.GenerativeModel('gemini-2.0-flash-exp')
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        logging.error(f"Error refining prompt with Gemini: {e}")
        raise

def analyze_images_with_gemini(image_urls):
    """Analyze images using Gemini AI."""
    IMAGE_PROMPT = (
        "Describe the dress in this image in extensive detail. Only return the words that you would use to search for the dress on a search website. Do not use commas, and only return one search."
    )
    try:
        analyzed_results = []
        model = genai.GenerativeModel('gemini-2.0-flash-exp')
        for img_url in image_urls:
            prompt = f"{IMAGE_PROMPT} Image URL: {img_url}"
            response = model.generate_content(prompt)
            analyzed_results.append(response.text.strip())
        return analyzed_results
    except Exception as e:
        logging.error(f"Error analyzing images with Gemini: {e}")
        raise

def fetch_shopping_results(analysis_results):
    """Fetch shopping results using SerpAPI for analyzed results."""
    if not SERPER_API_KEY:
        raise ValueError("SERPER_API_KEY is missing. Please provide a valid API key.")

    shopping_results = []
    try:
        for result in analysis_results:
            params = {
                "engine": "google_shopping",
                "q": result,
                "location": "Atlanta, Georgia, United States",
                "hl": "en",
                "gl": "us",
                "api_key": SERPER_API_KEY
            }
            search = GoogleSearch(params)
            results = search.get_dict()

            if "shopping_results" in results and results["shopping_results"]:
                for product in results["shopping_results"][:2]:
                    shopping_results.append({
                        "title": product["title"],
                        "price": product["price"],
                        "link": product["product_link"]
                    })
            else:
                logging.warning(f"No shopping results found for query: {result}")
    except Exception as e:
        logging.error(f"Error fetching shopping results: {e}")
        raise

    return shopping_results

def cache_images_in_pinecone(index, prompt_embedding, refined_prompt, image_urls, shopping_results):
    """Cache the generated images, metadata, and shopping results in Pinecone."""
    try:
        records = [{
            "id": str(uuid.uuid4()),
            "values": prompt_embedding,
            "metadata": {
                "prompt": refined_prompt,
                "images": json.dumps(image_urls),
                "shopping_results": json.dumps(shopping_results)
            },
        }]
        index.upsert(vectors=records)
        logging.info("Cached images, metadata, and shopping results in Pinecone successfully.")
    except Exception as e:
        logging.error(f"Error caching data in Pinecone: {e}")
        raise

def generate_image(prompt: str, gender: str, age: str):
    try:
        refined_prompt = refine_prompt_with_gemini(prompt, gender, age)
        logging.info(f"Refined Prompt: {refined_prompt}")

        index_name = "fashion" if gender == "Male" else "fashionfemale"
        selected_index = pinecone_client.Index(index_name)

        prompt_embedding = embeddings.embed_query(prompt)
        results = selected_index.query(vector=prompt_embedding, top_k=1, include_metadata=True)

        if results["matches"] and results["matches"][0]["score"] > 0.9:
            cached_images = json.loads(results["matches"][0]["metadata"]["images"])
            cached_shopping_results = json.loads(results["matches"][0]["metadata"].get("shopping_results", "[]"))
            return {"images": cached_images, "shopping_results": cached_shopping_results}

        num_variants = 5
        images = pipe([refined_prompt] * num_variants, num_inference_steps=25, height=384, width=384).images

        image_urls = []
        for img in images:
            filename = f"{uuid.uuid4()}.png"
            local_path = os.path.join(OUTPUT_DIR, filename)
            img.save(local_path)
            gcs_url = upload_to_gcs(local_path, filename)
            image_urls.append(gcs_url)

        gemini_analysis_results = analyze_images_with_gemini(image_urls[:2])

        shopping_results = fetch_shopping_results(gemini_analysis_results)

        cache_images_in_pinecone(selected_index, prompt_embedding, refined_prompt, image_urls, shopping_results)

        return {
            "images": image_urls,
            "gemini_analysis": gemini_analysis_results,
            "shopping_results": shopping_results
        }
    except Exception as e:
        logging.error(f"Error in generate_image: {e}")
        return {"error": str(e)}

if __name__ == "__main__":
    try:
        user_prompt = input("Enter the mood or occasion for which you want to generate your outfit designs: ").strip()

        # Display gender selection menu
        print("Select the gender for the outfit designs:")
        print("1. Male")
        print("2. Female")
        gender_choice = input("Enter your choice (1 or 2): ").strip()
        user_age = input("Enter Your age: ").strip()

        # Map user choice to gender
        if gender_choice == "1":
            user_gender = "Male"
        elif gender_choice == "2":
            user_gender = "Female"
        else:
            raise ValueError("Invalid choice. Please enter '1' for Male or '2' for Female.")

        if not user_prompt:
            raise ValueError("Outfit description cannot be empty.")

        result = generate_image(prompt=user_prompt, gender=user_gender, age = user_age)
        print(json.dumps(result, indent=4))

    except Exception as e:
        logging.error(f"An error occurred: {e}")



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Enter the mood or occasion for which you want to generate your outfit designs: Saturn
Select the gender for the outfit designs:
1. Male
2. Female
Enter your choice (1 or 2): 1
Enter Your age: 24


Token indices sequence length is longer than the specified maximum sequence length for this model (152 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['* " avant - garde male dress , saturnian rings , textural layers , dark palette , strong silhouette , flowing ." ** option with a bit more artistic flair :** * " cosmic male dress , saturn \'s orbit , dramatic draping , ethereal glow , sharp lines , statement ." i \'ve focused on using words that stable diffusion might understand well while keeping them concise .', '* " avant - garde male dress , saturnian rings , textural layers , dark palette , strong silhouette , flowing ." ** option with a bit more artistic flair :** * " cosmic male dress , saturn \'s orbit , dramatic draping , ethereal glow , sharp lines , statement ." i \'ve focused on using words that stable diffusion might understand well while ke

  0%|          | 0/25 [00:00<?, ?it/s]

{
    "images": [
        "https://storage.googleapis.com/fashion_designer/2a23a5a0-958e-42e4-a887-aa53245e391c.png",
        "https://storage.googleapis.com/fashion_designer/21c9d09d-1b9d-42a9-bca5-c530a6a053c5.png",
        "https://storage.googleapis.com/fashion_designer/5dc1a1f5-5ff6-4291-ba2f-0a127b523071.png",
        "https://storage.googleapis.com/fashion_designer/a0e99ae6-f100-455c-bbb4-ba1f1548d287.png",
        "https://storage.googleapis.com/fashion_designer/3086d3a1-cd7b-41d9-bb99-8b2fe19b668c.png"
    ],
    "gemini_analysis": [
        "lace midi fit and flare wedding dress",
        "lace midi dress puff sleeves"
    ],
    "shopping_results": [
        {
            "title": "Grace Loves Lace Jones Crepe Open Back Wedding Dress",
            "price": "$2,900.00",
            "link": "https://www.google.com/shopping/product/3662215624119381499?gl=us"
        },
        {
            "title": "June Bridals 1950s Vintage Graceful Floral V-Neck V-Back Two Piece Wedding Dre